In [1]:
!nvidia-smi

Mon Dec 16 01:49:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              1W /   80W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Installing YoloV8**

In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.49  Python-3.9.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
Setup complete  (16 CPUs, 15.7 GB RAM, 336.9/416.0 GB disk)


In [3]:
from ultralytics import YOLO

from IPython.display import display, Image
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Python SDK**

In [18]:
model = YOLO('yolov8n.pt')

# URL of the image
image_url = 'https://media.roboflow.com/notebooks/examples/dog.jpeg'

# Download the image from the URL
response = requests.get(image_url, stream=True)
if response.status_code == 200:
    image_data = np.asarray(bytearray(response.content), dtype="uint8")
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)  # Decode image
else:
    print("Failed to download the image")
    exit()


In [19]:
# Perform inference
results = model(image_path)


Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 c:\Users\Admin\Desktop\YoloV8\dog.jpeg: 640x384 1 person, 1 car, 1 dog, 62.3ms
Speed: 3.1ms preprocess, 62.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)


In [20]:
# Visualize and draw bounding boxes
for result in results[0].boxes:
    x1, y1, x2, y2 = result.xyxy[0].numpy()
    conf = result.conf.item()
    cls = int(result.cls.item())
    label = f"{model.names[cls]} {conf:.2f}"
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [23]:
# Save the resulting image
output_file = "processed_dog.jpeg"
cv2.imwrite(output_file, image)
print(f"Processed image saved as '{output_file}'")

Processed image saved as 'processed_dog.jpeg'


In [26]:
dir(results[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_keys',
 'boxes',
 'cpu',
 'cuda',
 'keypoints',
 'masks',
 'names',
 'new',
 'numpy',
 'obb',
 'orig_img',
 'orig_shape',
 'path',
 'plot',
 'probs',
 'save',
 'save_crop',
 'save_dir',
 'save_txt',
 'show',
 'speed',
 'summary',
 'to',
 'to_csv',
 'to_df',
 'to_json',
 'to_xml',
 'tojson',
 'update',
 'verbose']

In [1]:
import roboflow

In [2]:
roboflow.login()

visit https://app.roboflow.com/auth-cli to get your authentication token.


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="jsHxNU7I4CbmPVOcvxTJ")
project = rf.workspace("ahmedtech").project("football-player-detection-c8xfw")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Football-Player-Detection-1 in yolov8:: 100%|██████████| 1636/1636 [00:01<00:00, 1324.58it/s]


In [4]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

^C
